In [58]:
import glob

wav_files = sorted(glob.glob("../Data/MusicNet_Dataset/musicnet/musicnet/train_data/*.wav"))
csv_files = sorted(glob.glob("../Data/MusicNet_Dataset/musicnet/musicnet/train_labels/*.csv"))

# Đảm bảo số lượng file trùng khớp
assert len(wav_files) == len(csv_files), "Số lượng file WAV và CSV không khớp!"

In [60]:
from Model.CNN_3layer import CNN_3L
import torch.nn as nn
import torch.optim as optim
from Model.CNN_3L_pro import *
import torch
import os

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Khởi tạo mô hình, hàm mất mát và optimizer
model = CNN_3L().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

if os.path.exists("checkpoint.pth"):       
    checkpoint = torch.load("checkpoint.pth",weights_only=True, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Huấn luyện từng file một
for wav_path, csv_path in zip(wav_files, csv_files):
    print(f"\nfile : {wav_path}")
    # Load dữ liệu
    X_train, y_train = load_wav_csv(wav_path, csv_path)
    # Tạo DataLoader
    dataset = MusicDataset(X_train, y_train)
    train_loader = data.DataLoader(dataset, batch_size=64, shuffle=False)

    for epoch in range(20):  # Giảm số epoch cho từng file
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            print("Raw outputs:", outputs.max(), outputs.min(), outputs.mean())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        if epoch >= 19:
            print(f"Epoch {epoch+1}/20, Loss: {running_loss/len(train_loader):.4f}, "
              f"Accuracy: {100 * correct / total:.2f}%")

    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, "checkpoint.pth")
    print(f"Updated checkpoint sau file {wav_path}")

print("done training !")

In [ ]:
import torch
import librosa
import numpy as np

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Load mô hình đã huấn luyện
model = CNN_Pro().to(device)
checkpoint = torch.load("checkpoint.pth",weights_only=True, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
for name, param in model.named_parameters():
    if "fc2" in name:  # Lớp cuối cùng
        print(name, param.mean().item(), param.std().item())
model.eval()  # Chuyển sang chế độ dự đoán

def predict_notes(wav_path, sr=44100, hop_length=512, window_size=128, step=64):
    y, sr = librosa.load(wav_path, sr=sr)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, hop_length=hop_length)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    print("Mel spectrogram shape:", mel_spec_db.shape)
    print("Max value:", np.max(mel_spec_db), "Min value:", np.min(mel_spec_db))
    # Chia nhỏ spectrogram thành các đoạn như khi huấn luyện
    def create_windows(X, window_size, step):
        X_windows = []
        indices = []
        for i in range(0, X.shape[1] - window_size, step):
            X_windows.append(X[:, i:i + window_size])
            indices.append(i)
        return np.array(X_windows), indices

    X_test, indices = create_windows(mel_spec_db, window_size, step)
    X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1).to(device)

    # Dự đoán
    with torch.no_grad():
        outputs = model(X_test)
        predicted_notes = torch.argmax(outputs, dim=1).cpu().numpy()

    # Chuyển frame thành thời gian
    def frame_to_time(frame, sr, hop_length):
        return (frame * hop_length) / sr

    results = []
    prev_note = None
    start_time = None
    
    for idx, note in zip(indices, predicted_notes):
        time = frame_to_time(idx, sr, hop_length)
    
        if note != prev_note:  
            if prev_note is not None and prev_note != 128:  # Lưu nốt trước đó nếu nó không phải "không có nốt"
                results.append((prev_note, start_time, time))
            if note != 128:  # Khi đổi sang nốt mới, cập nhật start_time nếu không phải "không có nốt"
                start_time = time  
            prev_note = note  
    
    # Nếu file kết thúc với một nốt đang chơi, lưu lại nó
    if prev_note is not None and prev_note != 128:
        results.append((prev_note, start_time, time))

    return results

# Dùng model để dự đoán nốt nhạc của một file WAV
wav_file = "../Data/MusicNet_Dataset/musicnet/musicnet/train_data/2478.wav"
predicted_notes = predict_notes(wav_file)
print("Predicted notes:", predicted_notes)
print("Unique predicted values:", np.unique(predicted_notes))

for note, start, end in predicted_notes:
    if note == 128:  # Nếu là nhãn "không có nốt", bỏ qua
        continue
    note_name = librosa.midi_to_note(note)  # Chuyển số MIDI thành tên nốt nhạc
    print(f"Note: {note_name}, Start: {start:.2f}s, End: {end:.2f}s")

In [90]:
import librosa
import numpy as np

y, sr = librosa.load("../Data/MusicNet_Dataset/musicnet/musicnet/test_data/2628.wav")

# Phát hiện beat và downbeat
tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
onset_env = librosa.onset.onset_strength(y=y, sr=sr)

# Chuyển beats thành thời gian
beat_times = librosa.frames_to_time(beats, sr=sr)

if len(beat_times) > 1:
    # Tính khoảng cách giữa các beats
    beat_intervals = np.diff(beat_times)
    # Tìm khoảng cách phổ biến nhất giữa các beats mạnh
    avg_interval = np.median(beat_intervals)

    # Xác định nhịp theo độ lặp lại của downbeat
    if avg_interval < 0.4:  # Nếu khoảng cách nhỏ (thường là 3 beats trong 1 chu kỳ)
        time_signature = "3/4"
    else:  # Nếu khoảng cách lớn hơn (thường là 4 beats trong 1 chu kỳ)
        time_signature = "4/4"
else:
    time_signature = "Không xác định"

# In kết quả
print(f"Tempo: {tempo[0]:.2f} BPM")
print(f"Ước lượng nhịp: {time_signature}")

Tempo: 99.38 BPM
Ước lượng nhịp: 4/4


In [91]:
import librosa
import glob
import numpy as np

path = "../Data/MusicNet_Dataset/musicnet/musicnet/train_data/*.wav"

files = glob.glob(path)

unique_tempos = set()

for file in files:
    y, sr = librosa.load(file)
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    tempo = tempo[0] if tempo.ndim > 0 else tempo
    # Thêm tempo vào tập hợp
    unique_tempos.add(round(tempo, 2))

print("Các tempo duy nhất:")
for t in sorted(unique_tempos):
    print(f"{t}, ")

Các tempo duy nhất:
60.09, 
63.02, 
66.26, 
68.0, 
69.84, 
73.83, 
78.3, 
80.75, 
83.35, 
86.13, 
89.1, 
92.29, 
95.7, 
99.38, 
103.36, 
107.67, 
112.35, 
117.45, 
123.05, 
129.2, 
136.0, 
143.55, 
152.0, 
161.5, 
172.27, 
184.57, 
198.77, 
215.33, 
234.91, 


In [87]:
import mido
import glob

path = "../Data/MusicNet_Dataset/musicnet_midis/musicnet_midis/Bach/2302_fugue5.mid"

files = glob.glob(path)

unique_time_signatures = set()

for file in files:
    try:
        midi = mido.MidiFile(file, clip=True)  # clip=True giúp bỏ qua giá trị không hợp lệ
        for track in midi.tracks:
            for msg in track:
                if msg.type == 'time_signature':
                    time_sig = f"{msg.numerator}/{msg.denominator}"
                    unique_time_signatures.add(time_sig)
    except Exception as e:
        print(f"Lỗi khi đọc file {file}: {e}")

print("Các nhịp có trong dữ liệu MIDI:")
for ts in sorted(unique_time_signatures):
    print(ts)

Các nhịp có trong dữ liệu MIDI:
4/4
